In [ ]:
# SecondoMonitor -s     in Secondo/bin
#StartOptServer         in Secondo/Optimizer
#sgui                   in Secondo/Javagui

#\x00 = NULL
#\x04 = EOT (End of transmission)

In [ ]:
from IPython.core.debugger import set_trace
import os
import configparser as cfg
import pandas as pd
import socket
import sys
import time
from io import BytesIO
from struct import *
import asyncio
import ast
import nest_asyncio
nest_asyncio.apply()
binLists = True

In [ ]:
########## Nested List manipulation functions ############

#updates a specific element by value in a nested list with unknown depth and structure
def change(seq, what, make):
    for i, item in enumerate(seq):
        if item == what:
            seq[i] = make
        elif type(item) == list:
            change(item, what, make)
    return seq


#modifying the text types of form '****' to <text>****</text--->
def change_textform(seq):
    for i, item in enumerate(seq):
        if seq[i] and seq[i] != '<text>' and item[0] == "'" and item[1] != "(":
            seq[i] = "<text>" + item[1:]
            j = i+1 
            while True:  
      
                seq[i] += seq[j]
                del seq[j] 
                    
                if seq[j] == "'":
                    seq[i] += "</text--->"
                    del seq[j]
                    break
        
        elif seq[i] and seq[i] != '<text>' and item[0] == "'" and item[1] == "(":
            seq[i] = "<text>" + item[1:-1] + "</text--->"
                
        elif seq[i] and seq[i] == '<text>':
            item = ["<text>",seq[i+1],"</text--->"] 
            seq[i] = item
            del seq[i+1], seq[i+1]
            
        elif seq[i] and type(item) == list:
            change_textform(item)
        
    return seq


#flattens a nested list with unknown depth and structure to a flat list
def flatten_list(lisst):
    lst = []
    for i in lisst:
        if isinstance (i, list):
            lst.extend (flatten_list (i))
        else:
            lst.append (i)
    return lst  


#returns index of first appearance of item in a nested list with unknown depth and structure
def nested_index(lst, target):
    for index, item in enumerate(lst):
        if item == target:
            return [index]
        if isinstance(item, (list, tuple)):
            path = nested_index(item, target)
            if path:
                return [index] + path
    return []


#indices=[]
#for item in flattend_list:
#    indices.append(nested_index(nestedlist,item))
#print(indices)

#distinguishing the data types of nested list items
def NL_itemsTypeCheck(NL):
    flattend_list = flatten_list(NL)
    #print (flattend_list)
    for item in flattend_list:
        
        #if not item.startswith('"') and not item.startswith('<text>') and not item.startswith('</text--->') and item != '   ' and item.lower() != 'from' and item[-1] != ':' and item[-1] != '.' and item.lower() != "is":
        if item.isdigit() or item in ('True', 'False'):
            try:
                if type(eval(item)) in(int, float, bool):
                    change(NL, item, ast.literal_eval(item))
            except (ValueError, NameError):
                pass
    #print (NL)
    return(NL)


In [ ]:
# config_Datei lesen

class conn():
    
    def __init__(self,file=None,sec_srv=None,user=None,passw=None,sec_port=None,opt_srv=None,opt_port=None,opt_enable=None):
        if file and not (sec_srv and user and passw and sec_port and opt_srv and opt_port and opt_enable):
            self.file_name = file
            self.sec_srv = ""
            self.user = ""
            self.passw = ""
            self.sec_port = 0
            self.opt_srv = ""
            self.opt_port = 0
            self.opt_enable = ""
            self.startconnection = False
            self.Config = cfg.ConfigParser()
            self.get_from_file()
            
        elif not (file and sec_srv and user and passw and sec_port and opt_srv and opt_port and opt_enable):
            print("A needed parameter is missing!")
        elif not file and (sec_srv and user and passw and sec_port and opt_srv and opt_port and opt_enable):
            self.sec_srv = sec_srv
            self.user = user
            self.passw = passw
            self.sec_port = sec_port
            self.opt_srv = opt_srv
            self.opt_port = opt_port
            self.opt_enable = opt_enable
            self.startconnection = True
            
    def ConfigSectionMap(self, section):
        dict1 = {}
        options = self.Config.options(section)
        for option in options:
            try:
                dict1[option] = self.Config.get(section, option)
                if dict1[option] == -1:
                    DebugPrint("skip: %s" % option)
            except:
                print("exception on %s!" % option)
                dict1[option] = None
        return dict1

    def get_from_file(self):
        #dict1= ConfigSectionMap
        if os.path.isfile(os.getcwd() + '/' + self.file_name):
            self.Config.read(os.getcwd() + '/' + self.file_name)
            self.sec_srv = self.ConfigSectionMap("General")['servername']
            self.sec_port = self.ConfigSectionMap("General")['serverport']
            self.startconnection = self.ConfigSectionMap("General")['start_connection']
            self.user = self.ConfigSectionMap("General")['user']
            self.passw = self.ConfigSectionMap("General")['passwd']
            self.opt_srv = self.ConfigSectionMap("General")['optimizer_host']
            self.opt_port = self.ConfigSectionMap("General")['optimizer_port']
            self.opt_enable = self.ConfigSectionMap("General")['enable_optimizer']
    
        else:
            print("The ConfigFile not found!")    

        if self.opt_srv == "":
            self.opt_srv = "localhost"
        if int(self.opt_port) <= 0:
            self.opt_port = 1235
        
    def initialize(self):
        print(self.sec_srv, self.sec_port, self.startconnection, self.user, self.passw, self.opt_srv, self.opt_port, self.opt_enable)
        return self.sec_srv, self.sec_port, self.startconnection, self.user, self.passw, self.opt_srv, self.opt_port, self.opt_enable  

conn1 = conn("gui.cfg")
params = conn1.initialize()

In [ ]:
#Connecting to SecondoServer with acyncio method

class Secondo():
    def __init__(self, server, port):
        self.server = server
        self.port = port
        #self.loop = asyncLoop
        self.initialized = False
        self.message = "<Connect>\n" + params[3] + "\n" + params[4] + "\n" + "</Connect>\n"
        self.conn = None
        self.reader = None
        self.writer = None
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self.connect())
        #loop.close()
        #await connect()

    async def connect(self):
        conn = asyncio.open_connection(self.server, int(self.port))
        if conn:
            self.conn = conn
        reader, writer = await asyncio.wait_for(conn, timeout = 120)
        if reader and writer:
            self.reader, self.writer = reader, writer
        #reader, writer = await asyncio.open_connection(self.server, int(self.port))
    
        while True:
            line = await reader.readline()
            if not line:
                print("No response from Server!")
                self.initialized = False
                break
            line = line.decode()
            #.rstrip()
            if line != '<SecondoOk/>\n':
                print(f'Received: {line!r}')
                self.initialized = False
                break
            
            if line == '<SecondoOk/>\n':
                print(f'Received: {line}')
                print(self.message)
    
                writer.write(self.message.encode())
                await writer.drain()
    
                while True:
                    line = await reader.readline()
        
                    if not line:
                        print("Connection to Server failed!")
                        self.initialized = False
                        break

                    line = line.decode()
                    if line == '<SecondoIntro>\n':
                        print(f'Received: {line!r}')
                        while True:
                            line = await reader.readline()
        
                            if not line:
                                print("Connection to Server failed!")
                                self.initialized = False
                                break
                            line = line.decode()
                        
                            if line == '</SecondoIntro>\n':
                                print(f'Received: {line!r}')
                                self.initialized = True
                                print("Connection To Secondo Server established...")
                                #return reader, writer
                                break
                    
                            elif line == '<SecondoError>\n':
                                print(f'Received: {line!r}')
                                self.initialized = False
                                break
                            else:
                                print(f'Received: {line!r}')
                                self.initialized = False
                        break
            
            
                break    
        print(self.initialized)
    
    def get_Sec_conn(self):
        if self.conn:
            return self.conn
        else:
            return None
    
    def get_sec_streams(self):
        if self.reader and self.writer:
            return self.reader, self.writer
        else:
            None
        

#message = "<Connect>\n" + user + "\n" + passw + "\n" + "</Connect>\n"
sec = Secondo(params[0], params[1])
reader, writer = sec.get_sec_streams()
#reader, writer = async_loop.run_until_complete(sec.connect())
#async_loop.run_forever()

#cr = sec.connect()
#loop = asyncio.get_event_loop()
#reader, writer = loop.run_until_complete(cr)


In [ ]:
# connecting to OptimizerServer using asyncio methode

class Optimizer():
    
    def __init__(self, server, port, enable):
        self.server = server
        self.port = port
        self.enable = enable
        self.initialized = False
        
    #async def tcp_opt_client
    async def connect(self):
    
        if self.enable != "true":
            print("Optimizer Server cant not be contacted according to configurations.")
        elif self.initialized:
            print("Connection to Optimizer Server is already stablished.")
        elif self.enable == "true" and self.initialized == False:
            print("Starting the connection...")
            optreader, optwriter = await asyncio.open_connection(self.server, int(self.port))
            message = "<who>\n"
            
            while True:
                optwriter.write(message.encode("utf-8"))
                await optwriter.drain()
        
                print("Message sent to optserver...")
                print(message.encode("utf-8"))
    
                print("wants to read from optserver...")
                line = await optreader.readline()
                print("read from optserver...")
                
                line = line.decode("utf-8")
                print(f'Received: {line!r}')
           
                if not line:
                    print("No response from Server!")
                    self.initialized = False
                    break
            
                if line != "<optimizer>\n":
                    print(f'Received wrong data: {line!r}')
                    self.initialized = False
                    break
            
                if line == "<optimizer>\n":
                  
                    print(f'Received: {line!r}')
                    self.initialized = True
                    print("Connection to Optimizer stablished...")
                    return optreader, optwriter
                      
            print(self.initialized)
            
            
opt = Optimizer(params[5], params[6], params[7])
optreader, optwriter = await opt.connect()

#cr = opt.connect()
#loop = asyncio.get_event_loop()
#optreader, optwriter = loop.run_until_complete(cr)


In [ ]:
#Parsing Secondo_NestedList to a Python_Nestedlist


from pyparsing import *
import pprint
import re

def NLparse(content, ftype):  #ftype = True: file as argument, ftype = False: string as argument
    #ident = Word(alphas, alphanums + "-_.")
    #number = Word('-'+'+'+nums+'.')
    #nestedParens = nestedExpr('(', ')', content=enclosed) 
    enclosed = Forward()
    nestedParens = nestedExpr('(', ')')
    enclosed << (OneOrMore(nestedParens))
    if ftype:
        try:
            with open(content, encoding="utf8", errors='ignore') as content_file:
        
                content = content_file.read()
                NLresult = enclosed.parseString(content).asList()
                #print(NLresult)
                #return NLresult
        
        except (ParseException, ParseFatalException) as err:
            NLresult = []
            print(err)
            print("Invalid syntax at line {}, column {}: '{}': {}.".format(err.lineno,err.column,err.markInputline(),err.msg))

    if not ftype:
        
        try:
            NLresult = enclosed.parseString(content).asList()
            #print(NLresult)
            #return NLresult
        
        except (ParseException, ParseFatalException) as err:
            NLresult = []
            print(err)
            print("Invalid syntax at line {}, column {}: '{}': {}.".format(err.lineno,err.column,err.markInputline(),err.msg))

    
    NLresult = change_textform(NLresult)
    NLresult = NL_itemsTypeCheck(NLresult)
    return NLresult

    
    
#converts the python nested list to a string with nested parenthesis
def NLtoString(NL):
    strresult = pprint.pformat(NL)
    replacements = {"[": "(", "]": ")", "'": ""}
    strresult = "".join([replacements.get(c, c) for c in strresult])
    #print(strresult)
    return strresult
NL=NLparse("opt.txt", True)
#NL=NLparse("(restore database berlintest from berlintest)", False)
print(NL)



binLists = False
ftype = False

#returns the type of the given item of NestedList
def AtomType(item):
    
    if isinstance(item, str) and item[0] == '"':
        return "string"
    if isinstance(item, str) and item[0] != '"' and item[0] != '<':
        return "symbol"
    if isinstance(item, int):
        return "integer"
    if isinstance(item, float):
        return "real"
    if isinstance(item, bool):
        return "boolean"
    if (isinstance(item, list) and item[0] == "<text>") or (isinstance(item, str) and item.startswith('<text>')):
        return "text"
    if isinstance(item, list):
        return "list"


In [ ]:
async def readBinaryRecord():
    
    print("begin of readBinaryRecord().")
    LE = []
    Type = await reader.read(1)
    
    print(Type)
    print("Type: " + str(Type))
    Type_dict = {"BIN_LONGLIST": 0,
                    "BIN_INTEGER": 1,
                    "BIN_REAL": 2,
                    "BIN_BOOLEAN": 3,
                    "BIN_LONGSTRING": 4,
                    "BIN_LONGSYMBOL": 5,
                    "BIN_LONGTEXT": 6,
                    "BIN_LIST": 10,
                    "BIN_SHORTLIST": 11,
                    "BIN_SHORTINT": 12,
                    "BIN_BYTE":13,
                    "BIN_STRING": 14,
                    "BIN_SHORTSTRING": 15,
                    "BIN_SYMBOL": 16,
                    "BIN_SHORTSYMBOL": 17,
                    "BIN_TEXT": 18,
                    "BIN_SHORTTEXT": 19,
                    "BIN_DOUBLE": 20}
    typeCode = None
    for typ, val in Type_dict.items():
        if typ == Type:
            typeCode = val
            break
    if typeCode == 0: #LongList
        LE = []
        barray = bytearray(4) ####readInt()
        barray = await reader.readexactly(4)
        assert (int.from_bytes(barray, order = 'big', signed = True)) >= 0, "Invalid Integer-Byte read from Stream!"
        r = struct.unpack('>I',barray)
        ####
        if r == 0:
            return LE
        LE = await readBinaryRecord()
        Last = LE
        Next = []
        for i in range(r - 1):
            Next = await readBinaryRecord()
            if not Next:
                return None
            Last = Last.append(Next[0])
        return LE
            
    elif typeCode == 1:
        ####readInt()
        barray = bytearray(4)
        barray = await reader.readexactly(4)
        assert (int.from_bytes(barray, order = 'big', signed = True)) >= 0, "Invalid Integer-Byte read from Stream!"
        r = struct.unpack('>I',barray)
        ####
        LE[0] = r
        return LE 
        
    elif typeCode == 2:
        ####readReal()
        barray = bytearray(4)
        barray = await reader.readexactly(4)
        assert (int.from_bytes(barray, order = 'big', signed = True)) >= 0, "Invalid Real-Byte read from Stream!"
        r = struct.unpack('>f',barray)
        ####
        LE[0] = r
        return LE 
        
    elif typeCode == 3:
        ####readBool()
        barray = bytearray(1)
        barray = await reader.readexactly(1)
        assert (int.from_bytes(barray, order = 'big', signed = True)) >= 0, "Invalid Boolean-Byte read from Stream!"
        r = struct.unpack('?',barray)
        ####
        LE[0] = r
        return LE
        
            
    elif typeCode == 4:
        ####readLongString()
        barray = bytearray(4) #readInt()
        barray = await reader.readexactly(4)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid Int-Byte read from Stream!"
        len = unpack('>I',barray)
        barray = bytearray(len)
        barray = await reader.readexactly(len)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid LongSymbol-Byte read from Stream!"
        ###checking Environment.ENCODING not considered yet
        LE[0] = barray.decode()
        return LE
        
    elif typeCode == 5:
        ####readLongSymbol()
        barray = bytearray(4) #readInt()
        barray = await reader.readexactly(4)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid Int-Byte read from Stream!"
        len = unpack('>I',barray)
        barray = bytearray(len)
        barray = await reader.readexactly(len)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid LongSymbol-Byte read from Stream!"
        ###checking Environment.ENCODING not considered yet
        LE[0] = barray.decode()
        return LE
        
    elif typeCode == 6:
        ####readLongText()
        barray = bytearray(4) #readInt()
        barray = await reader.readexactly(4)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid Int-Byte read from Stream!"
        len = unpack('>I',barray)
        barray = bytearray(len)
        barray = await reader.readexactly(len)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid LongText-Byte read from Stream!"
        ###checking Environment.ENCODING not considered yet
        LE[0] = barray.decode()
        return LE
        
    elif typeCode == 10: #List
        LE = []
        ####readShort()
        barray = bytearray(2)
        barray = await reader.readexactly(2)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid ShortInt-Byte read from Stream!"
        r = unpack('>H',barray)
        ####
        if r == 0:
            return LE
        LE = await readBinaryRecord()
        Last = LE
        Next = []
        for i in range(r - 1):
            Next = await readBinaryRecord()
            if not Next:
                return None
            Last = Last.append(Next[0])
        return LE
        
    elif typeCode == 11: #ShortList
        LE = []
        ####readByte()
        barray = bytearray(1)
        barray = await reader.readexactly(1)
        assert (int.from_bytes(barray, order = 'big', signed = True)) >= 0, "Invalid Byte read from Stream!"
        r = int.from_bytes(barray, order = 'big', signed = False)
        ####
        if r == 0:
            return LE
        LE = await readBinaryRecord()
        Last = LE
        Next = []
        for i in range(r - 1):
            Next = await readBinaryRecord()
            if not Next:
                return None
            Last = Last.append(Next[0])
        return LE
        
    elif typeCode == 12:
        ####readShort()
        barray = bytearray(2)
        barray = await reader.readexactly(2)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid ShortInt-Byte read from Stream!"
        r = unpack('>H',barray)
        LE[0] = r
        return LE
    
    elif typeCode == 13:
        ####readByte()
        barray = bytearray(1)
        barray = await reader.readexactly(1)
        assert (int.from_bytes(barray, order = 'big', signed = True)) >= 0, "Invalid Byte read from Stream!"
        LE[0] = barray
        return LE
        
    elif typeCode == 14:
        ####readString()
        barray = bytearray(2)
        barray = await reader.readexactly(2)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid ShortInt-Byte read from Stream!"
        len = unpack('>H',barray)
        barray = bytearray(len)
        barray = await reader.readexactly(len)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid String-Byte read from Stream!"
        ###checking Environment.ENCODING not considered yet
        LE[0] = barray.decode()
        return LE
        
    elif typeCode == 15:
        ####readShortString()
        barray = bytearray(1)
        barray = await reader.readexactly(1)
        assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid Byte read from Stream!"
        len = int.from_bytes(barray, order = 'big', signed = False)
        barray = bytearray(len)
        barray = await reader.readexactly(len)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid ShortString-Byte read from Stream!"
        ###checking Environment.ENCODING not considered yet
        LE[0] = barray.decode()
        return LE
    
    elif typeCode == 16:
        ####readSymbol()
        barray = bytearray(2) #readShort()
        barray = await reader.readexactly(2)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid ShortInt-Byte read from Stream!"
        len = unpack('>H',barray)
        barray = bytearray(len)
        barray = await reader.readexactly(len)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid String-Byte read from Stream!"
        ###checking Environment.ENCODING not considered yet
        LE[0] = barray.decode()
        return LE
        
    elif typeCode == 17:
        ####readShortSymbol()
        barray = bytearray(1) #readByte()
        barray = await reader.readexactly(1)
        assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid Byte read from Stream!"
        len = int.from_bytes(barray, order = 'big', signed = False)
        barray = bytearray(len)
        barray = await reader.readexactly(len)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid ShortString-Byte read from Stream!"
        ###checking Environment.ENCODING not considered yet
        LE[0] = barray.decode()
        return LE
        
    elif typeCode == 18:
        ####readText()
        barray = bytearray(2) #readShort()
        barray = await reader.readexactly(2)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid ShortInt-Byte read from Stream!"
        len = unpack('>H',barray)
        barray = bytearray(len)
        barray = await reader.readexactly(len)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid Text-Byte read from Stream!"
        ###checking Environment.ENCODING not considered yet
        LE[0] = barray.decode()
        return LE
        
    elif typeCode == 19:
        ####readShortText()
        barray = bytearray(1) #readByte()
        barray = await reader.readexactly(1)
        assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid Byte read from Stream!"
        len = int.from_bytes(barray, order = 'big', signed = False)
        barray = bytearray(len)
        barray = await reader.readexactly(len)
        for b in barray:
            assert (int.from_bytes(b, order = 'big', signed = True)) >= 0, "Invalid ShortText-Byte read from Stream!"
        ###checking Environment.ENCODING not considered yet
        LE[0] = barray.decode()
        return LE
        
    elif typeCode == 20:
        ####readDouble()
        barray = bytearray(8)
        barray = await reader.readexactly(8)
        assert (int.from_bytes(barray, order = 'big', signed = True)) >= 0, "Invalid Double-Byte read from Stream!"
        r = struct.unpack('>d',barray)
        ####
        LE[0] = r
        return LE







###################################################


binLists = False
async def Receive_List(finalToken):
    print("begin of Receive_List()")
    answerList = []
    if not binLists:
        result = []
        count = 1
        print("binarylist = :" str(binLists))
        while True: 
            print("first line in Receive_List() read.")
            line = await reader.readline()
            print(str(count) + "th line read in Receive_List(): " + line)
            line = line.decode().rstrip()
            count += 1
            if finalToken in line:
                break
            if line:
                result.append(line + "\n")
                #result.append("\n")
            else:
                print("Network Error!")
                return None
            
        
        #answerList = result
        print("NLparse: \n")
        result = '(' + ' '.join(result) + ')'
        print(result)
        answerList = NLparse(result, False)
        return answerList[0]
    else: #binLists = True --> readBinaryFrom(inputStream)
        #readString()
        print("binarylist = True")
        line = await reader.read(3)
        signature = line.decode().rstrip()
        print("signature: " + signature)
        
        #readShort()
        barray = bytearray(2)
        barray = await reader.read(2)
        major = unpack('>H',barray)
        major = major[0]
        print("major: " + str(major))
        
        barray = await reader.read(2)
        minor = unpack('>H',barray)
        minor = minor[0]
        print("minor: " + str(minor))
        
        
        if signature != "bnl":
            print(signature + " is wrong Signatur!")
            return None
        if (major != 1 or (minor != 0 and minor != 1 and minor != 2)):
            print(str(major) + "." + str(minor) + " is wrong Version Number!")
            return None
        print("readBinaryRecords() is called.")
        answerList = await readBinaryRecord()
        return answerList


####################################################


async def Receive_Response():
    print("We r in Receive_Response() before reading.")
    line = await reader.readline()
    line = line.decode().rstrip()
    print("first line in Receive_Response() read:" + line)
    if not line:
        print("Network Error!")
        return None
    count = 2
    while line == "<Message>":
        print("Receive_List() called.")
        Message_List = await Receive_List("</Message>")
        if not Message_List:
            print("Error- MessageList is empty!")
            return None
        line = await reader.readline()
        line = line.decode().rstrip()
        print(str(count) + "th line in Receive_Response() read:" + line)
        count += 1
        if not line:
            print("Network Error!")
            return None
   
    if line == "<SecondoError>":
        line = await reader.readline()
        line = line.decode().rstrip()
        print(str(count) + "th line in Receive_Response() after <SecondoError> read:" + line)
        count += 1
        if line:
            print(line)
            line = await reader.readline()
            line = line.decode().rstrip()
            print(str(count) + "th line in Receive_Response() after <SecondoError> read:" + line)
            count += 1
        
        else:
            print("Error reading SecondoError!")
            return None
        return None
    
    if line != "<SecondoResponse>":
        print("Protocol Erroe!")
        return None
    
    print("Receive_List(</SecondoResponse>) is called.")
    answerList = await Receive_List("</SecondoResponse>")
    if not answerList:
        return None
    print(answerList)
    Error_Code = answerList[0]
    Error_Pos = answerList[1]
    Error_Message = answerList[2]
    Result_List = answerList[3]
    return Result_List

In [ ]:
#NLresult = []

binLists = False
ftype = False

#Command format: restore [database] database_name from <filename>  
async def restore_command(reader, writer, command):
    
    if reader and writer:
        if not command.startswith('(') :
            command = '(' + command.strip() + ')'
       
        CmdNL = NLparse(command, ftype)[0]
        print(CmdNL)
        length = len(CmdNL)
        if length != 4 and length != 5:
            print("restore_command(): command lenght wrong!") 
            return
        if length == 5: #restore database
            if CmdNL[0] != 'restore' or CmdNL[1] != 'database' or CmdNL[3] != 'from':
                print("restore_command: command wrong")  
                return
            name = CmdNL[2]
            filename = CmdNL[4]
            database = True
        if length == 4:
            if CmdNL[0] != 'restore' or CmdNL[2] != 'from':
                print("restore_command: command wrong")  
                return
            name = CmdNL[1]
            filename = CmdNL[3]
            database = False
        
        typ = AtomType(filename)
        if not typ in ("string", "symbol", "text"):
            print("restore_command: filename wrong")  
            return
        
        
        tag = "DbRestore" if database else "ObjectRestore"
        message = "<" + tag + ">\n"
        message = message.encode()
        writer.write(message)
        

        message = name + "\n"
        print("name:")
        print(name)
        message = message.encode()
        writer.write(message)
        
        
        message = "<FileData>\n"
        message = message.encode()
        writer.write(message)
        
        with open(filename, encoding="utf8", errors='ignore') as f:
       
            #flen = os.path.getsize(f.name)
            file = f.read()
            flen = len(file)
            print(flen)
            message = str(flen) + "\n"
            message = message.encode()
            writer.write(message)
            await writer.drain()
            
            #contents of file
            message = file
            message = message.encode()
            writer.write(message)
        
        
        message = "</FileData>\n"
        message = message.encode()
        #print(message)
        writer.write(message)
        
        message = "</" + tag + ">\n"
        message = message.encode()
        writer.write(message)
        await writer.drain()
        #print(message)
        
        #### secondoresponse- ReceiveResponse
        print("calling Receive_Response()")
        resp = await Receive_Response()
        print(resp)
        
        return resp
    
    else:
        print("reader and writer Sockets null")
        return None

    

async def perform_command(reader, writer, command):
    if command.lstrip().startswith('(restore') or command.lstrip().startswith('( restore') or command.lstrip().startswith('restore'):
        result = await restore_command(reader, writer, command)
        return result


command = '(restore database berlintest from berlintest)' 
#rs = async_loop.run_until_complete(perform_command(reader, writer, command))
rs = await perform_command(reader, writer, command)

#cr = perform_command(command)
#loop = asyncio.get_event_loop()
#rs = loop.run_until_complete(cr)

#print(rs)

In [ ]:
#finding the index of an item in a list
#index(...)
#L.index(value, [start, [stop]]) -> integer -- return first index of value

#
#i = iter(lisst)
#first = next(i)
#rest = list(i)


#recursive approach to itterate a nested list of unknown depth and structure
#def traverse(o, tree_types=(list, tuple)):
#    if isinstance(o, tree_types):
#        for value in o:
#            for subvalue in traverse(value, tree_types):
#                yield subvalue
#    else:
#        yield o

#a = number of inner lists
#lst = [[] for _ in range(a)]


In [ ]:
                                message = "<Secondo>\n"
                                message = message.encode()
                                writer.write(message)
                                
                                cmdLevel = "0" + "\n"
                                message = cmdLevel.encode()
                                writer.write(message)
        
        
                                message = "open databese berlintest\n"
                                message = message.encode()
                                writer.write(message)

                                
        
        
                                message = "</Secondo>\n"
                                message = message.encode()
                                writer.write(message)
                                await writer.drain()

                                print("start reading.")
                                line = await reader.readline()
                                line = line.decode().rstrip()
                                print(line)

In [ ]:
                                tag = "DbRestore"
                                message = "<" + tag + ">\n"
                                message = message.encode()
                                writer.write(message)
        
                                name = "berlintest"
                                message = name + "\n"
                                print("name:")
                                print(name)
                                message = message.encode()
                                writer.write(message)
        
        
                                message = "<FileData>\n"
                                message = message.encode()
                                writer.write(message)
                                
                                filename = "berlintest"
                                with open(filename, encoding="utf8", errors='ignore') as f:
       
                                    flen = os.path.getsize(f.name)
                                    print(flen)
                                    message = str(flen) + "\n"
                                    message = message.encode()
                                    writer.write(message)
                                    await writer.drain()
            
                                    #contents of file
                                    message = f.read()
                                    message = message.encode()
                                    writer.write(message)
        
        
                                message = "</FileData>\n"
                                message = message.encode()
                                #print(message)
                                writer.write(message)
        
                                message = "</" + tag + ">\n"
                                message = message.encode()
                                writer.write(message)
                                await writer.drain()
                                
                                print("Start reading")
                                #await asyncio.wait_for(reader.readline(),timeout=2)
                                line = await reader.readline()
                                if line:
                                    line = line.decode()
                                    print(line)
                                else:
                                    print("empty line!")
                                
                                print("End reading")

In [ ]:
from struct import *
unpack('>H',b'\x00\x13')


In [ ]:
int('\x13',16)